## LangSmith と Ragas を使ったオフライン評価

- オフライン評価では、LangSmith クライアント提供の「evaluate」関数を使う。
- 引数に推論の関数、Dataset の名前、Evaluator（評価器）を指定して evaluate 関数を実行すると、LangSmith の画面上で評価結果を確認できる。

### 利用可能な Evaluator（評価器）

- LangChain が提供する Evaluator では、評価用のプロンプトを使った LLM による評価、埋め込みベクトルの類似度やレーベンシュタイン距離による評価といった機能が提供されている。
- 独自定義関数も使用可能

#### ここでは、Ragas の評価メトリックを以下で使用

### Ragas の評価メトリクス

- RAG の評価軸：検索、生成、検索+生成
- Ragas では、これらに対して、評価メトリクスを提供

#### 「検索」の評価メトリクス：

- Context recall, Context precision, Context entity recall

#### 「生成」の評価メトリクス：

- Faithfulness, Answer relevancy

#### 「検索+生成」の評価メトリクス：

- Answer similarity, Answer correctness

#### 評価詳細：

##### -検索-

- Context precision - 質問と期待する回答を踏まえて、実際の検索結果のうち有用だと LLM で推論される割合（LLM 使用）
- Context recall - 期待する回答をいくつかの文章に分割したうち、実際の検索結果で説明できる割合（LLM 使用）
- Context entity recall - 期待する回答に含まれるエンティティ（物事）のうち、実際の検索結果で説明できる割合（LLM 使用）

##### -生成-

- Answer relevancy - 実際の回答が質問にどれだけ関連するか（実際の回答から LLM で推論した質問と、もとの質問の、埋め込みベクトルのコサイン類似度の平均値）（LLM 使用/Embedding 使用）
- Faithfulness - 実際の回答に含まれる主張のうち、実際の検索結果と一貫している割合（LLM 使用）

##### -検索+生成-

- Answer similarity - 実際の回答と期待する回答の、埋め込みベクトルのコサイン類似度（Embedding 使用）
- Answer correctness - 実際の回答と期待する回答の、事実的類似性と意味的類似性(Answer similarity)の加重平均（LLM 使用/Embedding 使用）

上記 Ragas の各評価メトリクスは、「期待する検索結果」を一切使わずに、実装されている。  
「期待する検索結果」がデータセットに含まれる場合は、以下の評価指標を使える

- Recall - 期待する検索結果のうち、実際の検索結果に含まれる割合
- Precision - 実際の検索結果のうち、期待する検索結果の割合


### カスタム Evaluator 実装

- カスタム Evaluator は、実際の実行結果（Run）と評価データ（Example）を引数として評価スコアを dict で返す関数として実装できる
- Ragas の評価メトリクスを使うときは、使用する LLM や Embedding モデルを設定する必要がある


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM

In [ ]:
class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        # ① 文脈文字列リストを 'retrieved_contexts' から作成
        contexts = [doc.page_content for doc in run.outputs["retrieved_contexts"]]

        # ② 必要なキー名に合わせて辞書を構築
        row = {
            # 'question' ではなく 'user_input'
            "user_input": example.inputs.get("user_input"),
            # 'answer' ではなく 'response'
            "response": run.outputs["response"],
            # そのまま 'retrieved_contexts'
            "retrieved_contexts": contexts,
            # 'ground_truth' ではなく 'reference'
            "reference": example.outputs.get("reference"),
        }

        # ③ スコア算出
        score = self.metric.score(row)
        return {"key": self.metric.name, "score": score}


# class RagasMetricEvaluator:
#     def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
#         self.metric = metric

#         # LLMとEmbeddingをMetricに設定
#         if isinstance(self.metric, MetricWithLLM):
#             self.metric.llm = LangchainLLMWrapper(llm)
#         if isinstance(self.metric, MetricWithEmbeddings):
#             self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

#     def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
#         context_strs = [doc.page_content for doc in run.outputs["contexts"]]

#         # Ragasの評価メトリクスのscoreメソッドでスコアを算出
#         score = self.metric.score(
#             {
#                 "question": example.inputs["question"],  # 質問
#                 "answer": run.outputs["answer"],  # 実際の回答
#                 "contexts": context_strs,  # 実際の検索結果
#                 "ground_truth": example.outputs["ground_truth"],  # 期待する回答
#             },
#         )
#         return {"key": self.metric.name, "score": score}


In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

In [39]:
metircs = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate for metric in metircs
]

### 推論関数の実装


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI
# from chromadb.config import Settings

In [40]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

prompt = ChatPromptTemplate.from_template("""
以下の文脈だけを踏まえて質問に回答してください。

文脈：'''
{context}
'''

質問：{question}
""")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

LangSmith での評価に使う推論関数は、データセットに保存した形式の dict を受け取り、実際の実行結果(Run)を dict して返す関数として実装する。  
データセットの入力から「question」を取り出して、推論（RAG などの処理）を行い、実際の検索結果や回答を返す関数は以下のようになる。


In [42]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    """
    Ragasの評価用に実行する推論関数。
    データセットの入力から「user_input」を取り出して推論し、
    必要なキー名で返却する。
    """
    # “question” ではなく “user_input” を使う
    user_input = inputs["user_input"]

    # RAG チェーンを実行
    output = chain.invoke(user_input)

    return {
        # “answer” ではなく “response”
        "response": output["answer"],
        # “contexts” ではなく “retrieved_contexts”
        "retrieved_contexts": output["context"],
        # もしデータセットに “reference” があるならそれも返す
        "reference": inputs.get("reference"),
    }


# def predict(inputs: dict[str, Any]) -> dict[str, Any]:
#     """
#     Ragasの評価用に実行する推論関数。
#     データセットの入力から「question」を取り出して、推論（RAGなどの処理）を行い、実際の検索結果や回答を返す。
#     """
#     question = inputs["question"]
#     output = chain.invoke(question)

#     return {
#         "answer": output["answer"],
#         "contexts": output["context"],
#     }


### オフライン評価の実装


In [43]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="test-dataset-2",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'tart-theory-87' at:
https://smith.langchain.com/o/5cb6609d-ce31-51cc-962f-6052d0aff4cc/datasets/f453892f-9c24-486b-9209-e651798b8cbf/compare?selectedSessions=b067fed9-1391-44f1-918b-46933270f057




0it [00:00, ?it/s]

C:\Users\yoshi\AppData\Local\Temp\ipykernel_15116\1225277247.py:29: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  score = self.metric.score(row)
C:\Users\yoshi\AppData\Local\Temp\ipykernel_15116\1225277247.py:29: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  score = self.metric.score(row)
C:\Users\yoshi\AppData\Local\Temp\ipykernel_15116\1225277247.py:29: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  score = self.metric.score(row)
C:\Users\yoshi\AppData\Local\Temp\ipykernel_15116\1225277247.py:29: DeprecationWarning: The function score was deprecated in 0.2, and will be removed in the 0.3 release. Use single_turn_ascore instead.
  score = self.metric.score(row)
C:\Users\yoshi\AppData\Local\Temp\ipykernel_15116\1225277247

,inputs.user_input,outputs.response,outputs.retrieved_contexts,outputs.reference,error,reference.response,reference.reference,reference.retrieved_contexts,feedback.context_precision,feedback.answer_relevancy,execution_time,example_id,id
0,How does LLM observability boost security in L...,LLM observability enhances security in Layerup...,[page_content='# Layerup Security\n\nThe [Laye...,None,None,The answer to given question is not present in...,The answer to given question is not present in...,[# PromptLayer\n\n>[PromptLayer](https://docs....,0.0,0.994580,2.440248,569b4ed4-bc0f-4f20-add7-c425959f379e,8e58a1db-1e9a-45ec-b468-bf57c1189571
1,How does MLflow simplify LLM provider interact...,MLflow simplifies interactions with various la...,[page_content='# MLflow AI Gateway for LLMs\n\...,None,None,MLflow simplifies LLM provider interactions by...,MLflow simplifies LLM provider interactions by...,[# MLflow Deployments for LLMs\n\n>[The MLflow...,1.0,0.828796,2.527124,3111f78c-231b-4c60-ba38-2b33cdf8b234,e40d0772-67f3-448f-99bb-f521cf619231
2,What is the purpose of creating an API token w...,The purpose of creating an API token when work...,[page_content='# PromptLayer\n\n>[PromptLayer]...,None,None,The purpose of creating an API token when work...,The purpose of creating an API token when work...,[# PromptLayer\n\n>[PromptLayer](https://docs....,1.0,0.998847,1.627504,0e656d1f-2674-4ec9-914a-83b7e82fdc20,7637ceb0-c322-4541-833e-af8a1307e747
3,What features are provided by the instant mess...,"Telegram provides several features, including:...",[page_content='# Telegram\n\n>[Telegram Messen...,None,None,Telegram provides features such as end-to-end ...,Telegram provides features such as end-to-end ...,[# Telegram\n\n>[Telegram Messenger](https://w...,1.0,0.903376,3.731595,c509ef6b-86e1-4ad8-9272-bd71fb2a00f9,ab0adfd7-254e-41b7-9703-2c871a671096


上記実行後、LangSmith に評価結果が保存される。  
LangSmith では、評価の１回１回を「Experiment」と呼ぶ。  
detaset の、「Experiment」タブから結果を見ることができる。
